In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import keras

In [ ]:
df = pd.read_csv('../input/diabetes/diabetes.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()
df.drop_duplicates(inplace=True)

In [ ]:
df.info()

In [ ]:
columns = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

for col in columns:
    df[col].replace(0, np.NaN, inplace=True)
    
df.info()    

In [ ]:
df.dropna(inplace=True)
df.info()

In [ ]:
X = df.drop('Outcome', axis=1)
X = StandardScaler().fit_transform(X)
y = df['Outcome']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=0)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

In [ ]:
model = Sequential()
model.add(Dense(8, activation = 'relu', input_shape = X_train[0].shape))
#model.add(Dropout(0.4))

model.add(Dense(8, activation='relu'))
#model.add(Dropout(0.4))

model.add(Dense(4, activation='relu')) 
#model.add(Dropout(0.4))

model.add(Dense(1, activation='sigmoid')) 
model.summary()

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer= opt ,loss='binary_crossentropy',metrics=['acc'])

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint('diabetes.h5', monitor='val_acc', mode='max', verbose=2, save_best_only=True)
history=model.fit(X_train, y_train, batch_size=16, epochs=350, validation_data=(X_test, y_test), callbacks=[checkpointer])

In [ ]:
present_model = keras.models.load_model('diabetes.h5')
present_model.summary()


In [ ]:
print("Accuracy of our model on test data : " , present_model.evaluate(X_test,y_test)[1]*100 , "%")

In [ ]:
# plot the training artifacts
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train_acc','val_acc'], loc = 'upper right')
plt.show()

In [ ]:
# plot the training artifacts

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss','val_loss'], loc = 'upper right')
plt.show()

In [ ]:
# predicting on test data.
pred_test = present_model.predict(X_test)
for i in range (len(pred_test)):
    if (pred_test[i] < 0.5):
        pred_test[i] = 0
    else:
        pred_test[i] = 1
pred_test = pred_test.astype(int)

In [ ]:

def view_result(array):
    array = np.array(array)
    for i in range(len(array)):
        if array[i] == 0:
            print("Not Diabetic")
        else:
            print("Diabetic")

In [ ]:
view_result(pred_test[:10])

In [ ]:
view_result(y_test[:10])